In [2]:
from mp_api.client import MPRester

with MPRester() as mpr:
    
    docs = mpr.materials.summary.search(
        chemsys=["Pb-Ti-O","Pb-Zr-O","Pb-Ti-Zr-O"],
        spacegroup_symbol="Pm-3m", #"P4mm" "R3mm"
        fields=['material_id', 'symmetry', 'structure']
    )

/ocean/projects/dmr110014p/ychiang4/.conda/envs/comm/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Retrieving SummaryDoc documents: 100%|██████████| 2/2 [00:00<00:00, 18978.75it/s]


In [4]:
# Modify default MP input set to enable berry phase and born charge calculation
# https://materialsproject.github.io/atomate2/user/codes/vasp.html#modifying-input-sets

from atomate2.vasp.sets.core import StaticSetGenerator
from atomate2.vasp.sets.mp import MPGGAStaticSetGenerator
from pymatgen.io.vasp.sets import MPStaticSet

from atomate2.vasp.jobs.core import StaticMaker
# from jobflow import run_locally
from jobflow_remote import submit_flow, set_run_config
from jobflow_remote.config.base import ExecutionConfig
import yaml

structure = docs[0].structure

# create a custom input generator set with a larger ENCUT
input_set_generator = MPStaticSet(
    user_incar_settings={
        "EDIFF": 1e-5,
        "LCALCPOL": True,
        "LCALCEPS": True,
        "IBRION": 5,
        "ISMEAR": 0
        # "ENCUT": 800
    }
)

# initialise the static maker to use the custom input set generator
static_maker = StaticMaker(
    input_set_generator=input_set_generator
)

# create a job using the customised maker
flow = static_maker.make(structure)

constraint = 'cpu'

# https://matgenix.github.io/jobflow-remote/user/tuning.html#submission
resources = {
    "job_name": "benchmark",
    "nodes": 1, 
    "ntasks_per_node": 64,
    "time": "00:30:00",
    "constraint": constraint,
    "partition": "RM",
    "qverbatim": "#SBATCH --gpu-bind=none\n#SBATCH --gpus=4" if constraint == 'gpu' else ""
}

atomate2_config_file = f"/jet/home/ychiang4/atomate2/config/atomate2-{resources['job_name']}-{resources['constraint']}.yaml"

with open(atomate2_config_file, 'w') as f:
    yaml.dump({
        "VASP_CMD": f"mpirun -np $SLURM_NTASKS /opt/packages/VASP/VASP6/6.4.3/GNU-SHMEM/vasp_std",
        "VASP_GAMMA_CMD": f"mpirun -np $SLURM_NTASKS /opt/packages/VASP/VASP6/6.4.3/GNU-SHMEM/vasp_gam"
    }, f)

config = ExecutionConfig(
    modules=['openmpi/5.0.3-gcc13.2.1', 'intel-mkl/2023.2.0', 'cuda/11.7.1'],
    export=dict(
        OMP_NUM_THREADS=1,
        OMP_PLACES="threads",
        OMP_PROC_BIND="spread",
        LD_LIBRARY_PATH="$LD_LIBRARY_PATH:/opt/packages/hdf5/hdf5-1.14.5/GNU/lib",
        ATOMATE2_CONFIG_FILE=atomate2_config_file,
    ),
)

print(submit_flow(
    flow, 
    # worker="local_shell",
    # worker='batch_worker', 
    project='comm',
    exec_config=config,
    resources=resources
))

# run_locally(static_job, create_folders=True)

['588']
